In [254]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sksparse.cholmod import cholesky as chol
from numpy.linalg import lstsq, cond
from scipy.optimize import broyden1
import mpmath
mpmath.mp.dps = 34

plt.close('all')

p1 = np.r_[0,0]
p2 = np.r_[1,0]
# p3 = np.r_[1/2,np.sqrt(3)/2]
p3 = np.r_[0,1]

m1 = (p2+p3)/2
m2 = (p1+p3)/2
m3 = (p1+p2)/2

b = (p1+p2+p3)/3

# vertices
T1 = np.c_[p1,p2,p3]
eval_T1 = lambda i,j : T1[0,:]**i*T1[1,:]**j

# edge midpoints
T2 = np.c_[m1,m2,m3]
eval_T2 = lambda i,j : T2[0,:]**i*T2[1,:]**j

# midpoint
T3 = np.c_[b]
eval_T3 = lambda i,j : T3[0,:]**i*T3[1,:]**j

# edge class
T4 = lambda a : np.c_[a*p1 + (1-a)*p2,
                      a*p2 + (1-a)*p1,
                      a*p3 + (1-a)*p1,
                      a*p1 + (1-a)*p3,
                      a*p3 + (1-a)*p2,
                      a*p2 + (1-a)*p3]

eval_T4 = lambda a,i,j : T4(a)[0,:]**(0*int(i<0)+i*int(i>=0))*T4(a)[1,:]**(0*int(j<0)+j*int(j>=0))

dT4 = np.c_[p1-p2,
            p2-p1,
            p3-p1,
            p1-p3,
            p3-p2,
            p2-p3]

eval_dT4 = lambda i,j : dT4[0,:]**(0*int(i<0)+i*int(i>=0))*dT4[1,:]**(0*int(j<0)+j*int(j>=0))

# inner class, type 1, TODO
T5 = lambda a : np.c_[a*m1 + (1-a)*p1,
                      a*m2 + (1-a)*p2,
                      a*m3 + (1-a)*p3]

dT5 = np.c_[m1-p1,
            m2-p2,
            m3-p3]

eval_T5 = lambda a,i,j : T5(a)[0,:]**(0*int(i<0)+i*int(i>=0))*T5(a)[1,:]**(0*int(j<0)+j*int(j>=0))

# inner class, type 2, TODO
T6 = lambda a,b : np.c_[b*(a*m1 + (1-a)*p1) + (1-b)*(a*m2 + (1-a)*p2),
                        b*(a*m1 + (1-a)*p1) + (1-b)*(a*m3 + (1-a)*p3),
                        b*(a*m3 + (1-a)*p3) + (1-b)*(a*m2 + (1-a)*p2),
                        b*(a*m2 + (1-a)*p2) + (1-b)*(a*m1 + (1-a)*p1),
                        b*(a*m3 + (1-a)*p3) + (1-b)*(a*m1 + (1-a)*p1),
                        b*(a*m2 + (1-a)*p2) + (1-b)*(a*m3 + (1-a)*p3)]

##### Evaluation #####

def integral(i,j):
    return math.factorial(j)*math.factorial(i)/math.factorial(i+j+2)

###### System ######

# order = 4
# Aij = lambda i,j,a: np.r_[sum(eval_T1(i,j)),
#                           sum(eval_T2(i,j)),
#                           sum(eval_T5(a,i,j))]

# order = 6
Aij = lambda i,j,a: np.r_[sum(eval_T1(i,j)),
                        #   sum(eval_T2(i,j)),
                          # eval_T3(i,j),
                          sum(eval_T4(a[0],i,j)),
                          sum(eval_T5(a[1],i,j)),
                          sum(eval_T5(a[2],i,j))]


# Aij = lambda i,j,a: np.r_[eval_T1(i,j),
#                           eval_T2(i,j),
#                           eval_T5(a,i,j)]

A = lambda a: np.c_[Aij(0,0,a),
                    
                    Aij(1,0,a),
                    # Aij(0,1,a),

                    # Aij(1,1,a),
                    Aij(2,0,a),
                    # Aij(0,2,a),

                    Aij(2,1,a),
                    # Aij(1,2,a),
                    Aij(3,0,a),
                    # Aij(0,3,a),

                    # Aij(2,2,a),
                    Aij(1,3,a),
                    # Aij(3,1,a),
                    Aij(4,0,a),
                    # Aij(0,4,a),

                    Aij(1,4,a),
                    # Aij(4,1,a),
                    Aij(2,3,a),
                    # Aij(3,2,a),
                    Aij(0,5,a),
                    # Aij(5,0,a),

                    Aij(1,5,a),
                    # Aij(5,1,a),
                    Aij(2,4,a),
                    # Aij(4,2,a),
                    # Aij(3,3,a),
                    Aij(0,6,a),
                    # Aij(6,0,a),
                    ]

rhs = np.r_[integral(0,0),
            
            integral(1,0),
            # integral(0,1),

            # integral(1,1),
            integral(2,0),
            # integral(0,2),

            integral(2,1),
            # integral(1,2),
            integral(3,0),
            # integral(0,3),

            # integral(2,2),
            integral(1,3),
            # integral(3,1),
            integral(4,0),
            # integral(0,4),

            integral(1,4),
            # integral(4,1),
            integral(2,3),
            # integral(3,2),
            integral(0,5),
            # integral(5,0),

            integral(1,5),
            # integral(5,1),
            integral(2,4),
            # integral(4,2),
            # integral(3,3),
            integral(0,6),
            # integral(6,0),
            ]

In [255]:
a = [0.1,0.2,0.2]
np.linalg.norm(A(a).T@lstsq(A(a)@A(a).T,A(a)@rhs, rcond=-1)[0]-rhs)

print(A([0.1,0.2,0.3]).shape, np.linalg.matrix_rank(A([0.1,0.2,0.3])))

(4, 13) 4


In [256]:
from numpy.linalg import solve,lstsq,cond
from scipy.optimize import broyden1,broyden2,diagbroyden,newton_krylov,minimize,root_scalar,root

f = lambda a : np.linalg.norm(A(a).T@lstsq(A(a)@A(a).T,A(a)@rhs, rcond=-1)[0]-rhs)

print("Cond: ", cond(A([0.7,0.3,0.4])))

# f = lambda a : np.linalg.norm(A(a).T@solve(A(a)@A(a).T,A(a)@rhs)-rhs)

# print(f(0.3,0.2,0.5))

def f_wrapper(args): return f(list(args))
print(f_wrapper([0.7,0.3,0.4]))

# stop

# num_a = broyden1(f, [0.1,0.1,0.1], f_tol = 1e-12, line_search = 'armijo')
num_a = minimize(f, [0.1,0.3,0.1], bounds = ((0, 1), (0, 1), (0, 1)), method='Nelder-Mead', tol = 1e-14, options={"maxiter" : 10_000, 'disp': True})
# num_a = root(f, x0 = [0.1,0.1,0.2], method='broyden1', tol = 1e-12)

print("kek ", num_a)
print(num_a.x[0])
print(1-num_a.x[0])
# print("Cond: ", cond(A(num_a)))

# aa = np.arange(0.2,0.5,0.001)

# import matplotlib.pyplot as plt

# faa = np.zeros(aa.size)
# for i,j in enumerate(aa):
#     faa[i] = f(j)

# plt.plot(aa,faa)

Cond:  476.41938232468465
0.00013567805194271944
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 301
         Function evaluations: 567
kek         message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 2.3121619881196204e-15
             x: [ 6.923e-01  2.372e-01  8.507e-01]
           nit: 301
          nfev: 567
 final_simplex: (array([[ 6.923e-01,  2.372e-01,  8.507e-01],
                       [ 6.923e-01,  2.372e-01,  8.507e-01],
                       [ 6.923e-01,  2.372e-01,  8.507e-01],
                       [ 6.923e-01,  2.372e-01,  8.507e-01]]), array([ 2.312e-15,  3.099e-15,  3.502e-15,  6.811e-15]))
0.6922540583666884
0.3077459416333116


In [243]:
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.7g" % x))

a = num_a.x
print(A(a).shape)
print(rhs.shape)

lstsq(A(a)@A(a).T,A(a)@rhs, rcond=-1)[0]

(4, 13)
(13,)


array([0.003565183, 0.01478472, 0.05094234, 0.08258971])

In [253]:
!pip install mpmath